In [ ]:
import functools as ft
import scipy.constants
import cmath
import numpy as np
import matplotlib.pyplot as plt
import kwant

import adaptive
adaptive.notebook_extension()
import holoviews as hv
import hpc05

import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import sns_system, spectrum

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(100, folder='two_dim_majoranas')
# hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(200, folder='two_dim_majoranas')

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

params_raw= dict(g_factor_middle = 26, g_factor_left = 0, g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20, alpha_left = 0, alpha_right = 0,
                 Delta_left = 2, Delta_right = 2,
                 B = 0, 
                 phase = 0,
                 T = 0.0,
                 V = 0)

params = dict(**constants,
              **params_raw)

# System definition

In [ ]:
syst_pars = dict(
    L_m = 200,
    L_x = 2400,
    L_sc_up = 200,
    L_sc_down = 200,
    z_x = 1200,
    z_y = 0,
    a   = 20,
    parallel_curve = False,
    sawtooth=False,
    transverse_soi = True,
    mu_from_bottom_of_spin_orbit_bands = True,
    k_x_in_sc = True,
    wraparound = False,
    current = False,
    ns_junction=False)

## Define additional dimensions

In [ ]:
dimension_values = dict(z_y = [0, 25, 50, 75, 100, 125, 150],
                        B    = np.linspace(0.5, 1.5, 11),
                        phase = [0, np.pi/2, np.pi],
                        system_shape = ['sawtooth', 'parallel_curve'])

dimension_combinations = adaptive.utils.named_product(**dimension_values)

In [ ]:
def cluster_calc_spectrum(momentum, syst_pars, params, k):
    syst_at_k, _, _ = sns_system.make_system(**syst_pars)
    e, ev = spectrum.calc_spectrum(syst_at_k, dict(**params, k_x=momentum), k=k)
    return np.sort(e)

#### Make function altering the parameters

In [ ]:
def get_altered_parameters(dimension_values, syst_pars, params):
    syst_pars_cpy = syst_pars.copy()
    params_cpy = params.copy()
    
    syst_pars_cpy['z_y'] = dimension_values['z_y']
    syst_pars_cpy[dimension_values['system_shape']] = True
    
    params_cpy['B'] = dimension_values['B']
    params_cpy['phase'] = dimension_values['phase']
    return syst_pars_cpy, params_cpy

#### Make learners

In [ ]:
learners= []
for combination in dimension_combinations:
    syst_pars_cpy, params_cpy = get_altered_parameters(combination, syst_pars, params)
    f = ft.partial(cluster_calc_spectrum, syst_pars=syst_pars, params=params, k=40)
    l_1D = adaptive.Learner1D(f, [0, np.pi])
    l_1D.fname = combination
    learners.append(l_1D)

bl = adaptive.BalancingLearner(learners)

In [ ]:
runner = adaptive.Runner(bl, ntasks=2)
runner.live_info()

In [ ]:
bl.

In [ ]:
bl.learners[0].fname

In [ ]:
bl.plot()